<a href="https://colab.research.google.com/github/FuadKhalit/TransferLearning_FER/blob/main/FER_TransferLearning_ConvNetVGGResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center>Ahmad Fuad Bin Khalit</center>
<center>TP058497</center>
<center>CT-100-3-M-DL</center>
<center>Asia Pacific University Malaysia</center>




<h1>Model Implementation</h1>


For Model implementation, the experiment will compared 3 model to find the best architecture to use for e-KYC system. The best model will be choose based on the latency, inference and accuracy. 

In [ ]:
#install plugin for tensorboard
!pip install -U tensorboard_plugin_profile # to install profile plugin in tensorboard
!pip install memory_profiler #to monitor memory usage of each line
!pip install line_profiler #to monitor time use

Mounting the Drive to colab

In [ ]:
#Mounting google drive to colab
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing all required Library to environment

In [ ]:
#Load Line Profiler extension
%load_ext line_profiler

#Load Memory Profiler extension
%load_ext memory_profiler

# Load the TensorBoard notebook extension
%load_ext tensorboard


In [ ]:
%time
#Import Python Library
import itertools
import datetime, os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import time



#Import Tensorflow Keras Library
import tensorflow as tf
import tensorflow.keras as keras
import keras.callbacks
from keras.callbacks import TensorBoard
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import Sequential
from keras import regularizers
from keras.layers import AveragePooling2D
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications import VGG16, VGG19, ResNet50
from keras.layers import Flatten, Dropout, Reshape, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import model_from_json
from tensorboard.plugins.hparams import api as hp
from keras.utils import  plot_model
from keras.applications.resnet50 import preprocess_input


* The Dataset link [Kaggle Competition: Emotion and Identity detection from face image](https://www.kaggle.com/c/facial-keypoints-detector)
* 2nd Dataset link [Challenges in Representation Learning: Facial Expression Recognition Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data)

In [ ]:
%time
# Read in the files from Google Drive

#train_df = pd.read_csv('/content/drive/MyDrive/Fuad_Assignment/train.csv')# 4178 dataset | 1st google drive acc
#train_df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Assignment2/FER2013.csv')# 28709 dataset | 1st google drive acc
#train_df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Assignment2/train.csv') # 4178 dataset | second google drive acc
train_df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Assignment2/FER2013.csv')# 28709 dataset | second google drive acc

train_df

Metadata Explaination

*   Source: https://www.kaggle.com/c/facial-keypoints-detector
*   Data are in csv file with 4178 image
*   input: 48x48 pixel grayscale (between 0 and 255)
*   target output:between 0 and 6: anger=0, disgust=1, fear=2, happy=3, sad=4, surprise=5, neutral=6)





# **Data Exploration**

use numpy to rebuild the csv value to image

In [ ]:
#check for imbalance of data
sns.countplot(x='emotion', data=train_df, palette='RdBu_r')


In [ ]:
%%time
# View an image #1
i = 2500
list_pixels = np.array([int(string) for string in train_df.iloc[i,1].split(' ')])
pixeled_image = list_pixels.reshape(48,48)
plt.imshow(pixeled_image, cmap='gray');

In [ ]:
# View an image #2
i = 1500
list_pixels = np.array([int(string) for string in train_df.iloc[i,1].split(' ')])
pixeled_image = list_pixels.reshape(48,48)
%timeit plt.imshow(pixeled_image, cmap='gray');

# **Preprocess data for base model: Convolution Layer CNN 1**

In [ ]:
#%%timeit
%%memit
# Reshape 1D array to 2D matrix for Conv2D

x_train = []
for i in range(len(train_df)):
  # convert string of pixels to list of integers
  list_pixels = [int(string) for string in train_df.iloc[i,1].split(' ')]
  # reshape 1D to 2D matrix
  pixeled_image = np.array(list_pixels).reshape(48,48)
  x_train.append(pixeled_image)

x = np.array(x_train)
y = train_df.iloc[:,0]
print (y)

In [ ]:
# Categorize y for compatiblity with softmax activation
from keras.utils import to_categorical
y = to_categorical(np.array(y))

In [ ]:
print(x.shape) # (4178, 48, 48)
y.shape # (4178, 7)

In [ ]:
# Train_test_split training data into training and test sets
x_train1, x_test, y_train1, y_test =  train_test_split(x, y, random_state = 123, test_size = 0.10)


In [ ]:
# Train_test_split training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, random_state = 123, test_size = 0.40)

x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, random_state = 123, test_size = 0.40)

In [ ]:
# Check shapes to ensure it's right
print(x_train.shape) # (3008, 48, 48)
print(x_test.shape) # (418, 48, 48)
print(y_train.shape) # (3008, 7)
print(y_test.shape) # (418, 7)
print(x_val.shape) # (752, 48,48)
print(y_val.shape) #(752, 7)

In [ ]:
# Reshape the data to be  (2924, 48, 48, 1) and (1254, 48, 48, 1) for x_train and x_val
# respectively. need to do this because input_shape in model_cnn.add needs a 2D image with 
# depth specified as 1
x_train_reshaped = np.expand_dims(x_train, axis = 3)
x_val_reshaped = np.expand_dims(x_val, axis = 3)
x_test_reshaped = np.expand_dims(x_test, axis = 3)


In [ ]:
print(x_train_reshaped.shape) #(3008, 48, 48, 1)
print(x_val_reshaped.shape) #(752, 48, 48, 1)
print(x_test_reshaped.shape) #(418, 48, 48, 1)

# **Train Base Layer : CNN 1**

---
This is the experiment basic architecture & using manual search for hyperparameter


In [ ]:
# Setting Hyperparameter (For Manual Search of Hyperparameter)
unit1 = 2048
unit2 = 128


# Define model as Sequential class

model_cnn = models.Sequential()

#Feature Extraction Layer
model_cnn.add(Conv2D(1, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                   input_shape=(48,48,1)))
model_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model_cnn.add(BatchNormalization(trainable=True))
model_cnn.add(layers.Dropout(0.3))#layer for larger dataset
model_cnn.add(Conv2D(32, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.3))#layer for larger dataset
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.3))#layer for larger dataset
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.3))#layer for larger dataset
#Fully Connected Layer
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(unit1, activation='relu'))
model_cnn.add(layers.Dropout(0.3))#layer for larger dataset
model_cnn.add(layers.Dense(unit2, activation='relu'))
model_cnn.add(layers.Dense(7, activation='softmax')) # 7 classifications
model_cnn.summary()


In [ ]:
#to stop overfitting
earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0.01, #minimum change for improvement
                              patience=30, # trigger when there is no improvement
                              verbose=0, mode='auto')

In [ ]:
model_cnn.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/CNN7/", histogram_freq=1, profile_batch='15,20')

history_cnn = model_cnn.fit(x_train_reshaped,
                        y_train,
                        epochs = 500,                  
                        batch_size=100,
                        validation_data=(x_val_reshaped, y_val),
                        callbacks=[tensorboard_callback,earlystop]
                       )

#print("Average test loss: ", np.average(history_cnn.history['loss'])



In [ ]:
#validation
#evaluate the model using test set
model_cnn.evaluate(x_test_reshaped,y_test)
model_cnn
#90/90 [==============================] - 0s 3ms/step - loss: 1.4656 - acc: 0.4375

In [ ]:
#Start TensorBoard within the notebook using magics
%tensorboard --logdir /content/logdir/CNN7/

Tensorboard Result:
The model run faster but the accuracy is too low 

# **Transfer Learning VGG Model**




**Data Preprocessing**

---

Data processing for the VGG model is different from our based model due to transfer learning takes RGB format for VGG model. The process are as below:

1.   Reshape train data(x) from 1D array to 2D matrix (using numpy array) & create label(y) from train data
2.   Convert numpy array to matrix for label (y)
3.   Split train data into train set & test set and their label (y)
4.   From train set, split it into train & validation set & their label (y)
5.   Reshaped all train, valid & test (x) by inserting new axis 
6.   Then use repeat function to flatten output



In [ ]:
#Clear Previous memory allocation
%reset -f

In [ ]:
# Reshape 1D array to 2D matrix for Conv2D

x_train = []
for i in range(len(train_df)):
  # convert string of pixels to list of integers and standarize data
  list_pixels = [int(string)/255 for string in train_df.iloc[i,1].split(' ')]
  # reshape 1D to 2D matrix
  pixeled_image = np.array(list_pixels).reshape(48,48)
  x_train.append(pixeled_image)

x = np.array(x_train)
y = train_df.iloc[:,0]

In [ ]:
# Categorize y to y category for compatiblity with softmax activation
from keras.utils import to_categorical
y = to_categorical(np.array(y))

In [ ]:
print(x.shape) # (4178, 48, 48)
y.shape # (4178, 7)
(4178, 48, 48)

In [ ]:
# Train_test_split training data into training and test sets
x_train1, x_test, y_train1, y_test =  train_test_split(x, y, \
                                          random_state = 123, test_size = 0.10)

In [ ]:
# Create train_test_split for validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, random_state = 123, test_size = 0.40)

In [ ]:
# Check shapes to ensure it's right
print(x_train.shape) # (3008, 48, 48)
print(x_test.shape) # (418, 48, 48)
print(y_train.shape) # (3008, 7)
print(y_test.shape) # (418, 7)
print(x_val.shape) # (752, 7)
print(y_val.shape)

In [ ]:
# Reshape the data to be  (2924, 48, 48, 1) and (1254, 48, 48, 1) for x_train and x_val
# respectively. need to do this because input_shape in model_cnn.add needs a 2D image with 
# depth specified as 1
x_train_reshaped = np.expand_dims(x_train, axis = 3)
x_val_reshaped = np.expand_dims(x_val, axis = 3)
x_test_reshaped = np.expand_dims(x_test, axis = 3)

In [ ]:
# Need to repeat our 48x48x1 to 3 dimensions, because transfer learning models take RGB format
#For VGG Model
x_train_rgb = np.repeat(x_train_reshaped, 3, axis=3)
x_val_rgb = np.repeat(x_val_reshaped, 3, axis=3)
x_test_rgb = np.repeat(x_test_reshaped, 3, axis=3)



In [ ]:
print(x_train_rgb.shape) #(3008, 48, 48, 3)
print(x_val_rgb.shape) #(752, 48, 48, 3)
print(x_test_rgb.shape) #(418, 48, 48, 3)
print(x_train_rgb.dtype)

**Build & Run The Pre-trained Model**

---

Using keras pretrain VGG16 architecture. The model will be cut the top layer which contain its fully connected layer (FC) from its original model and will be replace with facial emotion detection FC. NO modification was made on this architecture as we'll use hyperparameter search & data input pipeline to optimized from this basic pretrained model.

In [ ]:
#Experimenting with tuning the Pretrained Layer

vgg_base = VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3))

#set freeze & unfreeze layer
#vgg_base.trainable = False #freeze all so gradient descend wont modify them
vgg_base.trainable = True #unfreeze specified layer to let backprop tweak them to increase performance (Rule: >train data >layer can unfreeze )

print("Number of layers in the base model: ", len(vgg_base.layers))

# Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
fine_tune_at = 17 # backprop will tweak layer 17 to 19

# Freeze all the layers before the `fine_tune_at` layer
for layer in vgg_base.layers[:fine_tune_at]:
  layer.trainable =  False
vgg_base.summary()

In [ ]:
# # Define model as Sequential class (Small Dataset)
# model_vgg = models.Sequential()
# model_vgg.add(vgg_base)
# model_vgg.add(layers.GlobalAveragePooling2D())
# model_vgg.add(layers.Dense(128, activation='relu'))
# model_vgg.add(BatchNormalization(momentum=0.5))
# model_vgg.add(layers.Dropout(0.3))
# model_vgg.add(layers.Dense(7, activation='softmax')) # 7 classifications
# model_vgg.summary()    

In [ ]:
# Define model as Sequential class (Large Dataset)
model_vgg = models.Sequential()
model_vgg.add(vgg_base)
model_vgg.add(BatchNormalization(momentum=0.5, trainable=True))
model_vgg.add(layers.GlobalAveragePooling2D())
model_vgg.add(layers.Dense(1028, activation='tanh'))
model_vgg.add(BatchNormalization(momentum=0.5, trainable=True))
model_vgg.add(layers.Dropout (0.4))
model_vgg.add(layers.Dense(128, activation='tanh'))
model_vgg.add(BatchNormalization(momentum=0.5, trainable=True))
model_vgg.add(layers.Dropout (0.4))
model_vgg.add(layers.Dense(16, activation='tanh'))
model_vgg.add(BatchNormalization(momentum=0.5, trainable=True))
model_vgg.add(layers.Dropout (0.4))
model_vgg.add(layers.Dense(7, activation='softmax')) # 7 classifications
model_vgg.summary()    

In [ ]:
model_vgg.compile(loss='categorical_crossentropy',
                  optimizer="adam",
                  metrics=['acc'])

In [ ]:
#to stop overfitting
earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0.01, #minimum change for improvement
                              patience=10, # trigger when there is no improvement
                              verbose=0, mode='auto')

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/vgg1.9/", histogram_freq=1, profile_batch='15,20')

In [ ]:
history_vgg = model_vgg.fit(x_train_rgb,
                    y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val_rgb, y_val), 
                    callbacks = [tensorboard_callback, earlystop])

#Experiment 1 (Freeze all layer)
#Epoch 15/15
#301/301 [==============================] - 5s 16ms/step - loss: 0.9009 - acc: 0.6729 - val_loss: 1.0564 - val_acc: 0.6064
#experiment 2 (unfreeze layer 16-19)
#Epoch 15/15
#301/301 [==============================] - 7s 22ms/step - loss: 0.2477 - acc: 0.9132 - val_loss: 0.6415 - val_acc: 0.7766
#experiment 3 (unfreeze layer 3-19)
#Epoch 15/15
#301/301 [==============================] - 12s 40ms/step - loss: 0.0851 - acc: 0.9727 - val_loss: 0.4739 - val_acc: 0.8750
#Summary of experiment
# the more layer we unfreeze the higher the accuracy but in expense of speed of the training


In [ ]:
#evaluate the model using test set
model_vgg.evaluate(x_test_rgb,y_test)
model_vgg

#freeze all Layer
#14/14 [==============================] - 0s 13ms/step - loss: 0.9762 - acc: 0.7010
#unfreeze at 16th layer
#14/14 [==============================] - 0s 13ms/step - loss: 0.7975 - acc: 0.8182
#unfreeze 3rd layer
#14/14 [==============================] - 0s 12ms/step - loss: 0.5753 - acc: 0.8995

In [ ]:
%tensorboard --logdir /content/logdir/vgg1.9/

Tensorboard Analysis:

---
training set loss function was low but the validation set is high. This show that there are overfitting problem in the model.


In [ ]:
14/14 [==============================] - 0s 7ms/step - loss: 1.2188 - acc: 0.5478

# **Input PipeLine Experiment (VGG16)**
This will maximize throughput of training session
Input Pipeline = ETL Process : 

1.   Extract (extract the data from in memory to data pipeline)
2.   Transform (normalize, transform the data)
3.   Load (load to training model)





In [ ]:
#change the dataset type from float64 to float32 (for performance advantage as tensorflow GPU are design for 32bit number)
x_train_rgb =tf.cast(x_train_rgb, tf.float32)

In [ ]:
#extract dataset into Tensorflow tf.data (API for input pipeline)
train_ds = tf.data.Dataset.from_tensor_slices((x_train_rgb, y_train)).batch(512)
valid_ds =tf.data.Dataset.from_tensor_slices((x_val_rgb, y_val)).batch(512)

In [ ]:
# START THE TRANSFORMING PROCESS
#function to normalize image 
def normalize_img (image, label):
  return tf.cast(image, tf.float32)/255.0, label

In [ ]:
#Automatic the tuning parameter in data pipeline
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_auto = train_ds.map(normalize_img,num_parallel_calls=AUTOTUNE)# Normalization the data function using map function 
train_auto = train_ds.cache() #caches the dataset after normalizing the images
train_auto = train_ds.shuffle(buffer_size= 100) # to create randomness in dataset so to avoid bias when training the model
train_auto = train_ds.batch(batch_size=64)# number of image batch into data pipeline
train_auto = train_ds.prefetch(buffer_size = AUTOTUNE) #to alter the preprocessing execution
valid_auto = valid_ds.map(normalize_img,num_parallel_calls=AUTOTUNE)
valid_auto = valid_ds.batch(batch_size=64)
valid_auto = valid_ds.prefetch(buffer_size= AUTOTUNE)

In [ ]:
# Instantiate a base model with pre-trained weights
vgg_base = VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3))

In [ ]:
# Define model as Sequential class using default transfer learning setting
model_vggip = models.Sequential()
model_vggip.add(vgg_base) # load the pre-trained base model (Load Process)
model_vggip.add(layers.Flatten())
model_vggip.add(layers.Dense(128, activation='tanh'))
model_vggip.add(layers.Dense(64, activation='tanh'))
model_vggip.add(layers.Dense(7, activation='softmax')) # 7 classifications
model_vggip.summary()

In [ ]:
#Configures the model for training
model_vggip.compile(loss='categorical_crossentropy',
                  optimizer="sgd",
                  metrics=['acc'])

In [ ]:
#Setting up Tensorboard callback parameter
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/vgg_IP/", histogram_freq=1, profile_batch='15,20')

In [ ]:
#Trains the model for a fixed number of epochs (iterations on a dataset).
history_cnn = model_vggip.fit(train_auto,
                    epochs=15,
                    validation_data=valid_auto, 
                    callbacks = [tensorboard_callback])

In [ ]:
#run the tensorboard to monitor data input pipeline profile (to see any improvement on step time speed, GPU optimizing)
%tensorboard --logdir /content/logdir/vgg_IP/

# **HyperParameter GridSearch VGG16**

---
Identify the best hyperparameters for the model
Inspired by https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams & https://medium.com/ml-book/neural-networks-hyperparameter-tuning-in-tensorflow-2-0-a7b4e2b574a1

In [ ]:
#Hyperparameter 1st Run
HP_FREEZE = hp.HParam('fine_tune_at', hp.Discrete([5, 9, 16])) 
HP_NUM_UNITS = hp.HParam('num_units2', hp.Discrete([256,512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_vgg').as_default():
  hp.hparams_config(
    hparams=[HP_FREEZE,HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
# # Experiment setup by choosing hyperparameter for tuning
# HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

# METRIC_ACCURACY = 'accuracy'

# #Log experiment configuration to tensorboard
# with tf.summary.create_file_writer('logs/hparam_tuningvgg16').as_default():
#   hp.hparams_config(
#     hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
#     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#   )

In [ ]:
def train_test_resnetmodel(hparams):
    resnet_base = VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3))
    resnet_base.trainable = True #False = Freeze all | True = Train Selected Layer

    #Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
    fine_tune_at = hparams[HP_FREEZE] # backprop will tweak layer 20 to 175

# Freeze all the layers before the `fine_tune_at` layer
    for layer in resnet_base.layers[:fine_tune_at]:

        layer.trainable =  False

#Adapt TensorFlow run & log it into tensorboard 
def train_test_model(hparams):
  vgghp_model = tf.keras.models.Sequential([tf.keras.applications.VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3)),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='tanh'),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Dense(7, activation=tf.nn.softmax), # 7 classifications
                                    ])
          
  vgghp_model.compile(optimizer=hparams[HP_OPTIMIZER],
                loss='categorical_crossentropy',
                metrics=['accuracy'],
  )
  
  history = vgghp_model.fit(train_auto,epochs=15)
  validation_data=(valid_auto),  
 
  return history.history['accuracy'][-1]

In [ ]:
# #Adapt TensorFlow run & log it into tensorboard 
# def train_test_model(hparams):
#   vgghp_model = tf.keras.models.Sequential([tf.keras.applications.VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3)),
#                                       tf.keras.layers.Flatten(),
#                                       tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='tanh'),
#                                       tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
#                                       tf.keras.layers.Dense(7, activation=tf.nn.softmax), # 7 classifications
#                                     ])
          
#   vgghp_model.compile(optimizer=hparams[HP_OPTIMIZER],
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'],
#   )
  
#   history = vgghp_model.fit(train_auto,epochs=50)
#   validation_data=(valid_auto),  
 
#   return history.history['accuracy'][-1]

In [ ]:
#For each run, log an hparams summary with the hyperparameters and final accuracy:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    accuracy= tf.reshape(tf.convert_to_tensor(accuracy), []).numpy()
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=10)

In [ ]:
#Start run & log them
session_num = 0
for fine_tune_at in HP_FREEZE.domain.values:
  for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
      for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
          HP_FREEZE: fine_tune_at,
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_vgg/' + run_name, hparams)
      session_num += 1

In [ ]:
#Accessing Tensorboard hyperparameter plugin
%tensorboard --logdir logs/hparam_vgg

# **Recalibrate VGG Hyperparameter**

---
Combination : Freeze layer (5), Dense layer(512), dropout (0.1),


In [ ]:
#Experimenting with tuning the Pretrained Layer

vgg_base = VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3))

#set freeze & unfreeze layer
#vgg_base.trainable = False #freeze all so gradient descend wont modify them
vgg_base.trainable = True #unfreeze specified layer to let backprop tweak them to increase performance (Rule: >train data >layer can unfreeze )

print("Number of layers in the base model: ", len(vgg_base.layers))

# Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
fine_tune_at = 5 # backprop will tweak layer 16 to 19

# Freeze all the layers before the `fine_tune_at` layer
for layer in vgg_base.layers[:fine_tune_at]:
  layer.trainable =  False
vgg_base.summary()

In [ ]:
# Define model as Sequential class
# model_vggbest = tf.keras.models.Sequential([tf.keras.applications.VGG16(include_top= False, weights='imagenet', input_shape = (48,48,3)),
#                                       tf.keras.layers.Flatten(),
#                                       tf.keras.layers.Dense(512, activation='tanh'),
#                                       tf.keras.layers.Dropout(0.1),
#                                       tf.keras.layers.Dense(7, activation=tf.nn.softmax), # 7 classifications
#                                     ])
          

In [ ]:
model_vggbest = models.Sequential()
model_vggbest.add(vgg_base)
model_vggbest.add(BatchNormalization(momentum=0.5, trainable=True))
model_vggbest.add(layers.GlobalAveragePooling2D())
model_vggbest.add(layers.Dense(512, activation='tanh'))#Densee layer 512 best hyperparameter
model_vggbest.add(BatchNormalization(momentum=0.5, trainable=True))
model_vggbest.add(layers.Dropout (0.1)) #dropout 0.1 best hyperparameter
model_vggbest.add(layers.Dense(128, activation='tanh'))
model_vggbest.add(BatchNormalization(momentum=0.5, trainable=True))
model_vggbest.add(layers.Dropout (0.1))
model_vggbest.add(layers.Dense(16, activation='tanh'))
model_vggbest.add(BatchNormalization(momentum=0.5, trainable=True))
model_vggbest.add(layers.Dropout (0.1))
model_vggbest.add(layers.Dense(7, activation='softmax')) # 7 classifications


In [ ]:
# # Define model as Sequential class using default transfer learning setting
# model_vggbest = models.Sequential()
# model_vggbest.add(vgg_base) # load the pre-trained base model (Load Process)
# model_vggbest.add(layers.GlobalAveragePooling2D())
# model_vggbest.add(layers.Dense(512, activation='relu'))
# model_vggbest.add(layers.Dropout(0.1))
# model_vggbest.add(layers.Dense(64, activation='relu'))
# model_vggbest.add(layers.Dropout(0.1))
# model_vggbest.add(layers.Dense(7, activation='softmax')) # 7 classifications
# #model_vggip.summary()

In [ ]:
model_vggbest.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.5), #0.01 is the best learning rate based on hyperparameter tuning,
                      metrics=['acc'])

In [ ]:
#to stop overfitting
earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0.01, #minimum change for improvement
                              patience=10, # trigger when there is no improvement
                              verbose=0, mode='auto')

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/vggbest/", histogram_freq=1, profile_batch='15,20')

In [ ]:
history_vgg = model_vggbest.fit(train_auto,
                    epochs=50,
                    #batch_size=10,
                    validation_data=(valid_auto), 
                    callbacks = [tensorboard_callback, earlystop])

#save all model
model_vggbest.save('/content/drive/MyDrive/Fuad_Assignment/vgg_model(All).h5')# save in hdfs format
tf.keras.models.save_model(model_vggbest,'/content/drive/MyDrive/Fuad_Assignment/vgg_tfmodel(All)')# save in Tensorflow format  
#save weight only
model_vggbest.save_weights('/content/drive/MyDrive/Fuad_Assignment/vgg_model(weight)v1.h5')  
#save architecture only
json_string=model_vggbest.to_json() 


In [ ]:
#evaluate the model using test set
model_vggbest.evaluate(x_test_rgb,y_test)
model_vggbest

In [ ]:
%tensorboard --logdir /content/logdir/vggbest/

# **RESNET50 Model**

Preprocess for Resnet Model

In [ ]:
#Clear Previous memory allocation
%reset -f

In [ ]:
# Reshape 1D array to 2D matrix for Conv2D

x_train = []
for i in range(len(train_df)):
  # convert string of pixels to list of integers and standarize data
  list_pixels = [int(string)/255 for string in train_df.iloc[i,1].split(' ')]
  # reshape 1D to 2D matrix
  pixeled_image = np.array(list_pixels).reshape(48,48)
  x_train.append(pixeled_image)

x = np.array(x_train)
y = train_df.iloc[:,0]

In [ ]:
# Categorize y to y category for compatiblity with softmax activation
from keras.utils import to_categorical
y = to_categorical(np.array(y))

In [ ]:
# print(x.shape) # (4178, 48, 48)
# y.shape # (4178, 7)
# (4178, 48, 48)

In [ ]:
# Train_test_split training data into training and test sets
x_train_r, x_test_r, y_train_r, y_test_r =  train_test_split(x, y, \
                                          random_state = 123, test_size = 0.15)

In [ ]:
# Create train_test_split for validation sets
x_trainRN, x_valRN, y_trainRN, y_valRN = train_test_split(x_train_r, y_train_r, random_state = 123, test_size = 0.30)

In [ ]:
# Check shapes to ensure it's right
print(x_trainRN.shape) # (17081, 48, 48)
print(x_test_r.shape) # (4307, 48, 48)
print(y_trainRN.shape) # (17081, 7)
print(y_test_r.shape) # (4307, 7)
print(x_valRN.shape) # (7321, 48,48)
print(y_valRN.shape) #(7321,7)

In [ ]:
# Reshape the data to be  (2924, 48, 48, 1) and (1254, 48, 48, 1) for x_train and x_val
# respectively. need to do this because input_shape in model_cnn.add needs a 2D image with 
# depth specified as 1
x_train_reshaped5 = np.expand_dims(x_trainRN, axis = 3)
x_val_reshaped5 = np.expand_dims(x_valRN, axis = 3)
x_test_reshaped5 = np.expand_dims(x_test_r, axis = 3)

In [ ]:
# Need to repeat our 48x48x1 from 1 channel to 3 channel, because transfer learning models take RGB format
#For Resnet Model
x_train_rgb50 = np.repeat(x_train_reshaped5, 3, axis=3)
x_val_rgb50 = np.repeat(x_val_reshaped5, 3, axis=3)
x_test_rgb50 = np.repeat(x_test_reshaped5, 3, axis=3)

In [ ]:
print(x_train_rgb50.shape) #(3008, 48, 48, 3)
print(x_val_rgb50.shape) #(752, 48, 48, 3)
print(x_test_rgb50.shape) #(418, 48, 48, 3)
print(x_train_rgb50.dtype)

In [ ]:
#Note:- For Training Model (x_train_rgb50, y_trainRN) Validation (x_val_rgb50, y_valRN) Testing (x_test_rgb50, y_test_r)

In [ ]:
#set batch for dataset


Freeze & Unfreeze Layer for Resnet50

In [ ]:
#Experimenting with tuning the Pretrained Layer

resnet_base = ResNet50(include_top= False, weights='imagenet', input_shape = (48,48,3))

#set freeze & unfreeze layer
#resnet_base.trainable = False #freeze all so gradient descend wont modify them
#resnet_base.trainable = True #unfreeze specified layer to let backprop tweak them to increase performance (Rule: >train data >layer can unfreeze )

print("Number of layers in the base model: ", len(resnet_base.layers))

# Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
#fine_tune_at = 20 # backprop will tweak layer 100 to 175

# Freeze all the layers before the `fine_tune_at` layer
#for layer in resnet_base.layers[fine_tune_at:]:
#  layer.trainable =  False
#resnet_base.summary()

#for layer in resnet_base.layers:
#   if isinstance(layer, keras.layers.BatchNormalization):
#     layer.trainable = True
#   else:
#     layer.trainable = False
#resnet_base.summary()

In [ ]:
resnet_base.trainable = True #False = Freeze all | True = Train Selected Layer

#Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
fine_tune_at = 172 # backprop will tweak layer 20 to 175

# Freeze all the layers before the `fine_tune_at` layer
for layer in resnet_base.layers[:fine_tune_at]:
   layer.trainable =  False



**Due to small dataset, Data Augmentation is required in order for the model generalize better**

In [ ]:
#data augmentation
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [ ]:
# #Experiment with top layer #1
# x = resnet_base.output
# x = data_augmentation(x)
# x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dense(128,activation='tanh')(x)
# x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# x = keras.layers.Dense(64, activation='tanh')(x)
# # A Dense classifier with a multiple unit (multiple classification)
# outputs = keras.layers.Dense(7, activation='softmax')(x)
# model = keras.Model(resnet_base.input, outputs)
# plot_model(model)
# Epoch 100/100
# #32/32 [==============================] - 5s 170ms/step - loss: 1.7273 - acc: 0.2987 - val_loss: 1.7301 - val_acc: 0.3051

In [ ]:
#experiment with top layer #2 #best Top Layer
x = resnet_base.output
#x = data_augmentation(x)
#x = keras.layers.BatchNormalization()(x)
x = keras.layers.BatchNormalization(momentum=0.5, trainable=True)(x)
#x = keras.layers.Activation('relu')(x)
#x = keras.layers.GlAveragePooling2D(pool_size=2)(x)
y = keras.layers.GlobalAveragePooling2D()(x)
#y = keras.layers.Flatten()(x)
#y = keras.layers.Dense(512, activation='relu')(y)
y = keras.layers.Dense(512, activation='relu')(y)
y = keras.layers.BatchNormalization(momentum=0.5, trainable=True)(y)
y = keras.layers.Dropout(0.1)(y)
y = keras.layers.Dense(128, activation='relu')(y)
y = keras.layers.BatchNormalization(momentum=0.5, trainable=True)(y)
y = keras.layers.Dropout(0.1)(y)
y = keras.layers.Dense(32, activation='relu')(y)
y = keras.layers.BatchNormalization(momentum=0.5, trainable=True)(y)
y = keras.layers.Dropout(0.1)(y)
# y = keras.layers.BatchNormalization()(y)
# y = keras.layers.Dropout(0.5)(y)
outputs = keras.layers.Dense(7, activation='softmax')(y)

#instantiate model
model = keras.Model(resnet_base.input, outputs)
plot_model(model)

# #Epoch 100/100
# #33/33 [==============================] - 6s 180ms/step - loss: 1.4303 - acc: 0.4418 - val_loss: 1.6767 - val_acc: 0.3475

In [ ]:
model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(0.01),
                  metrics=['acc'])

In [ ]:
#to stop overfitting
earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0.01, #minimum change for improvement
                              patience=10, # trigger when there is no improvement
                              verbose=0, mode='auto')

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/resnet/", histogram_freq=1, profile_batch='15,20')

In [ ]:
history_res = model.fit(x_train_rgb50,
                    y_trainRN,
                    epochs=500,
                    batch_size=647, #total train = 20670/647 = 32 each step 
                    validation_data=(x_val_rgb50, y_valRN), 
                    callbacks = [tensorboard_callback, earlystop])

#Experiment 1 (Freeze all layer)
#Epoch 15/15
#301/301 [==============================] - 7s 25ms/step - loss: 1.7564 - acc: 0.3341 - val_loss: 1.6664 - val_acc: 0.3830
#experiment 2 (unfreeze at layer 160)
#Epoch 100/100
#
#experiment 3 (unfreeze at layer 20)
#Epoch 100/100
#301/301 [==============================] - 10s 34ms/step - loss: 0.7092 - acc: 0.7763 - val_loss: 0.8134 - val_acc: 0.7420
#Summary of experiment
#Perform much better if unfreeze the layer for backprop to tweak the weight


In [ ]:
#evaluate the model using test set
model.evaluate(x_test_rgb50,y_test_r)
model

In [ ]:
%tensorboard --logdir /content/logdir/resnet/

# **Input PipeLine Experiment (ResNet50)**

---

As in real life deep learning problem will handle more larger set that will not fit in the memory, we will emulate of building data pipeline that extract data from distributed system. For this we use tensorflow input pipeline. we will used ETL principle:
* Extract (extract the data from in memory to data pipeline)
* Transform (normalize, transform the data)
* Load (load to training model)

**Transformation** \\
Below are the transformation technique in this experiment to transform data using tf input pipeline 
* Shuffle - Prevent data from remembering pattern from training data that will make the model unable to generalize well
* Batch - for very large dataset, processing small chunk of data at time will make 
* Prefetch - prefetching increace efficentcy of accelerator (GPU/TPU) by preventing pipeline waiting for preprocessing by CPU to finish. its prefetch the data ahead of time to GPU to avoid valuable cycle doing nothing.
* Interleave - Interleave is similar to shuffle function except its shuffle the instance to avoid data learning from ordering of the data. Then there is AUTOTUNE which we let tensorflow to automatically choose the value dynamically at runtime.

**Load** \\
Finally after the transformation stage complete, we feed the data into the model







In [ ]:
#change the dataset type from float64 to float32 (lower the bit faster the process but effect accuracy)
x_train_rgb50 = tf.cast(x_train_rgb50, tf.float32)
x_val_rgb50 = tf.cast (x_val_rgb50, tf.float32)
x_test_rgb50 = tf.cast (x_test_rgb50, tf.float32)

In [ ]:
#extract dataset into Tensorflow tf.data (API for input pipeline)
train_ip1 = tf.data.Dataset.from_tensor_slices((x_train_rgb50, y_trainRN)).batch(256)
valid_ip1 =tf.data.Dataset.from_tensor_slices((x_val_rgb50, y_valRN)).batch(256)

In [ ]:
# START THE TRANSFORMING PROCESS
#function to normalize image 
def normalize_img (image, label):
  return tf.cast(image, tf.float32)/255.0, label

In [ ]:
#Automatic the tuning parameter in data pipeline
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_auto1 = train_ip1.map(normalize_img,num_parallel_calls=AUTOTUNE)# Normalization the data function using map function 
train_auto1 = train_ip1.cache() #caches the dataset after normalizing the images
train_auto1 = train_ip1.shuffle(buffer_size= 100) # to create randomness in dataset so to avoid bias when training the model
train_auto1 = train_ip1.batch(batch_size=512)# number of image batch into data pipeline
train_auto1 = train_ip1.prefetch(buffer_size = AUTOTUNE) #to alter the preprocessing execution
valid_auto1 = valid_ip1.map(normalize_img,num_parallel_calls=AUTOTUNE)
valid_auto1 = valid_ip1.batch(batch_size=128)
valid_auto1 = valid_ip1.prefetch(buffer_size= AUTOTUNE)

## **Feed the tensor to model**

In [ ]:
resnet_ip = ResNet50(include_top= False, weights='imagenet', input_shape = (48,48,3))

In [ ]:
resnet_ip.trainable = True

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

x= resnet_base.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(128,activation='tanh')(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
x = keras.layers.Dense(64, activation='tanh')(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(7, activation='softmax')(x)
model_ip1 = keras.Model(resnet_base.inputs, outputs)
model_ip1.summary()

In [ ]:
model_ip1.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.SGD(1e-5),
                  metrics=['acc'])

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/resnet_IP/", histogram_freq=1, profile_batch='15,20')

In [ ]:
history_cnn = model_ip1.fit(train_auto1,
                    epochs=15,
                    validation_data=valid_auto1,
                    callbacks = [tensorboard_callback])

In [ ]:
%tensorboard --logdir /content/logdir/resnet_IP/

# **HyperParameter GridSearch ResNet50**

---

Tuned inspired by https://medium.com/ml-book/neural-networks-hyperparameter-tuning-in-tensorflow-2-0-a7b4e2b574a1

From previous experiment, we find out that our model very slow to converge. This raise the question of
* Which learning rate is the fast to converge?
* Which layer to unfreeze to give best result ?
* How many unit our dense layer need to perform?
* What dropout rate is the best to avoid overfitting? 

Instead of guessing which parameter work best. we use grid search to find best combination



---
From running grid search we found that 
* learning rate 0.01 is better than 0.1
* unfreeze middle layer give more accuracy than bottom or top (88 better than  120 and 30
* 512 dense unit is the best
* Drop rate 0.2 is better than 0.3
the best accuracy of this combination for 15 epoch is 0.43145

From the result of 1st run, refinement will be made on the hyperparameter. we will test as follow:-
* will learning rate 0.001 better than current best (0.01)?
* will unfreeze layer 73 and 103 (+-15 from current best 88) give better result
* we are satisfied with 512 dense unit so no more refinement
* Will drop rate 01 is better than current best (0.2) 

Best combination (Unfreeze at 88 layer, Dense Unit (88), Dropout (10) LR(0.01)
Accuracy after 15 epoch = 0.465

In [ ]:
# #Hyperparameter 1st Run
# HP_FREEZE = hp.HParam('fine_tune_at', hp.Discrete([58, 88, 120])) 
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([256,512]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2,0.3))
# HP_LR = hp.HParam('learning_rate', hp.RealInterval(0.01, 0.1))

# METRIC_ACCURACY = 'accuracy'

# with tf.summary.create_file_writer('logs/hparam_tuningResNet50').as_default():
#   hp.hparams_config(
#     hparams=[HP_FREEZE,HP_NUM_UNITS2, HP_DROPOUT, HP_LR],
#     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#   )

In [ ]:
#Hyperparamater 2nd Run

HP_FREEZE = hp.HParam('fine_tune_at', hp.Discrete([73, 88, 103])) 
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.2))
HP_LR = hp.HParam('learning_rate', hp.RealInterval(0.001, 0.01))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuningResNet50_2nd').as_default():
  hp.hparams_config(
    hparams=[HP_FREEZE,HP_NUM_UNITS2, HP_DROPOUT, HP_LR],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
#exp new dataset
def train_test_resnetmodel(hparams):
    resnet_base = ResNet50(include_top= False, weights='imagenet', input_shape = (48,48,3))
    resnet_base.trainable = False #False = Freeze all | True = Train Selected Layer

    #Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
    fine_tune_at = hparams[HP_FREEZE] # backprop will tweak layer 20 to 175

# Freeze all the layers before the `fine_tune_at` layer
    for layer in resnet_base.layers[:fine_tune_at]:

        layer.trainable =  False

    data_augmentation = keras.Sequential(
        [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )

     
    x = resnet_base.output
    x = data_augmentation(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.AveragePooling2D(pool_size=2)(x)
    y = keras.layers.Flatten()(x)
    y = keras.layers.Dense(hparams[HP_NUM_UNITS2], activation='relu')(y)
    y = keras.layers.BatchNormalization()(y)
    y = keras.layers.Dropout(hparams[HP_DROPOUT])(y)
    outputs = keras.layers.Dense(7, activation='softmax')(y)

    #instantiate model
    res50_hp = keras.Model(resnet_base.input, outputs)
          
    res50_hp.compile(optimizer=keras.optimizers.SGD(learning_rate=hparams[HP_LR]),#here
                loss='categorical_crossentropy',
                metrics=['accuracy'],
    )
  
    history = res50_hp.fit(train_auto1,epochs=15)
    validation_data=(valid_auto1),  
    #val_accuracy = model.evaluate(x_val_reshaped, y_val),
  

    return history.history['accuracy'][-1]

In [ ]:
# HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([128, 256]))
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([32,64]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2,0.3))
# HP_LR = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd',]))

# METRIC_ACCURACY = 'accuracy'

# with tf.summary.create_file_writer('logs/hparam_tuningResNet50').as_default():
#   hp.hparams_config(
#     hparams=[HP_NUM_UNITS1,HP_NUM_UNITS2, HP_DROPOUT, HP_OPTIMIZER],
#     metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
#   )

In [ ]:
# def train_test_resnetmodel(hparams):
#     data_augmentation = keras.Sequential(
#     [
#         layers.experimental.preprocessing.RandomFlip("horizontal"),
#         layers.experimental.preprocessing.RandomRotation(0.1),
#     ]
#     )

     
#     x = resnet_base.output
#     x = keras.layers.GlobalAveragePooling2D()(x)
#     x = keras.layers.Dense(hparams[HP_NUM_UNITS1],activation='tanh')(x)
#     x = keras.layers.Dropout(hparams[HP_DROPOUT])(x)# Regularize with dropout
#     x = keras.layers.Dense(hparams[HP_NUM_UNITS2], activation='tanh')(x)      
#     # A Dense classifier with a single unit (binary classification)
#     outputs = keras.layers.Dense(7, activation='softmax')(x)
#     res50_hp = keras.Model(resnet_base.input, outputs)
          
#     res50_hp.compile(optimizer=hparams[HP_OPTIMIZER],#here
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'],
#     )
  
#     history = res50_hp.fit(train_auto1,epochs=15)
#     validation_data=(valid_auto1),  
#     #val_accuracy = model.evaluate(x_val_reshaped, y_val),
  

#     return history.history['accuracy'][-1]

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_resnetmodel(hparams)
    accuracy= tf.reshape(tf.convert_to_tensor(accuracy), []).numpy()
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=10)

In [ ]:
session_num = 0

for fine_tune_at in HP_FREEZE.domain.values:
  for num_units2 in HP_NUM_UNITS2.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
      for learning_rate in (HP_LR.domain.min_value, HP_LR.domain.max_value):
        hparams = {
            HP_FREEZE: fine_tune_at,
            HP_NUM_UNITS2: num_units2,
            HP_DROPOUT: dropout_rate,
            HP_LR: learning_rate,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuningResNet50_2nd/' + run_name, hparams)
        session_num += 1

#result
#512 unit is the best
#lower dropout good result (0.2 is better than 0.3)
#learning rate lower the better (0.01 is better than 0.1)
#unfreeze upper level more accurate (120 is better than 88 than 58)

In [ ]:
# session_num = 0

# for num_units1 in HP_NUM_UNITS1.domain.values:
#   for num_units2 in HP_NUM_UNITS2.domain.values:
#     for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#       for optimizer in HP_OPTIMIZER.domain.values:
#         hparams = {
#             HP_NUM_UNITS1: num_units1,
#             HP_NUM_UNITS2: num_units2,
#             HP_DROPOUT: dropout_rate,
#             HP_OPTIMIZER: optimizer,
#         }
#         run_name = "run-%d" % session_num
#         print('--- Starting trial: %s' % run_name)
#         print({h.name: hparams[h] for h in hparams})
#         run('logs/hparam_tuningResNet50/' + run_name, hparams)
#         session_num += 1

In [ ]:
#Accessing hyperparameter plugin
%tensorboard --logdir logs/hparam_tuningResNet50_2nd

# **Recalibrate Resnet Model with Best Hyperparameter**
using test dataset



---
Best Architecture: ResNet50
HyperParameter: Fine tune at (88), Dense layer I(512), learning rate(0.01), Dropout (0.1)

---
Based on the best combination of hyperparameter, we retrain our model. we then monitor the performance of the model closely. once we spotted gradient explotion on the validation loss, we interrupt the kernel and tweak the top layer by adding more dense layer. If the validation loss behave erraticly i.e. the loss increase exponentially then sudden decrease, we adjust the learning rate to be more slower than previous.

the training epoch is set up to 500 epoch as we not unable to predict when the model will converge.



In [ ]:
#input_t = keras.Input(shape=(48,48,3))
resnet_base = ResNet50(include_top= False, weights='imagenet', input_shape= (48,48,3))
resnet_base.trainable = True

#Fine-tune from this layer onwards (make sure to unfreeze layer first before finetune)
fine_tune_at = 88 # backprop will tweak layer 88 to 175 based on best hyperparameter tuning
# Freeze all the layers before the `fine_tune_at` layer
for layer in resnet_base.layers[:fine_tune_at]:
   layer.trainable =  False

In [ ]:
# x= resnet_base.output
# x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dense(256,activation='tanh')(x)
# x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
# # x = keras.layers.Dense(128, activation='tanh')(x)
# # x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
# x = keras.layers.Dense(64, activation='tanh')(x)

# # A Dense classifier with a single unit (binary classification)
# outputs = keras.layers.Dense(7, activation='softmax')(x)
# modelx = keras.Model(resnet_base.inputs, outputs)
# modelx.summary()



In [ ]:
# x= resnet_base.output
# x = keras.layers.GlobalAveragePooling2D()(x)
# #x = keras.layers.Dense(1024,activation='relu')(x)
# #x= keras.layers.BatchNormalization()(x)
# #x = keras.layers.Dropout(0.3)(x)  # Regularize with dropout
# x = keras.layers.Dense(512, activation='relu')(x)
# #x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Dropout(0.5)(x)
# #x = keras.layers.Dense(512, activation='relu')(x)
# #x = keras.layers.BatchNormalization()(x)
# x = keras.layers.Dense(64, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)
# # A Dense classifier with a single unit (binary classification)
# outputs = keras.layers.Dense(7, activation='softmax')(x)
# modelx = keras.Model(resnet_base.inputs, outputs)


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

x = resnet_base.output
x = data_augmentation(x)  
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.AveragePooling2D(pool_size=2)(x)
y = keras.layers.Flatten()(x)
y = keras.layers.Dense(512, activation='relu')(y)  #512 unit is the best based on hyperparameter tuning
y = keras.layers.BatchNormalization()(y)
y = keras.layers.Dense(512, activation='relu')(y)
y = keras.layers.BatchNormalization()(y)
y = keras.layers.Dropout(0.1)(y) #0.1 is the best based on hyperparameter tuning
outputs = keras.layers.Dense(7, activation='softmax')(y)

#instantiate model
modelx = keras.Model(resnet_base.input, outputs)

In [ ]:
#add Learning rate scheduler & momentum to speed up convergence
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

modelx.compile(loss='categorical_crossentropy', 
                  optimizer=keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.1), #0.01 is the best learning rate based on hyperparameter tuning
                  metrics=['acc'])

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logdir/resnet_bestmodelv1/", histogram_freq=1, profile_batch='150,160')

In [ ]:
history_res = modelx.fit(train_auto1,
                    epochs=100, #Run large iteration, interrupt when gradient vector become tiny number as gradient descent already reach global minima
                    validation_data=(valid_auto1),
                    callbacks = [tensorboard_callback]) 

#save all model
modelx.save('/content/drive/MyDrive/Fuad_Assignment/resnet_model(All)v1.h5')# save in hdfs format
tf.keras.models.save_model(modelx,'/content/drive/MyDrive/Fuad_Assignment/resnet_tfmodel(All)v1')# save in Tensorflow format  
#save weight only
modelx.save_weights('/content/drive/MyDrive/Fuad_Assignment/resnet_model(weight)v1.h5')  
#save architecture only
json_string=modelx.to_json() 

# The model seem to face Curse of Dimensionality 

In [ ]:
#evaluate the model using test set
modelx.evaluate(x_test_rgb50,y_test_r)
modelx
#135/135 [==============================] - 3s 16ms/step - loss: 2.7775 - acc: 0.4829

In [ ]:

%tensorboard --logdir /content/logdir/resnet_bestmodelv1/

# **Testing the Model on Random Image**


---
Final part of the experiment is testing outside experiment environment for evaluating real time performance against some random image. this is to validate that our training models can performed well against unseen data that not come from the same sources. 


In [ ]:
#Clear Previous memory allocation
%reset -f

In [ ]:
#Load train model
from keras.models import load_model
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Fuad_Assignment/resnet_tfmodel(All)')

In [ ]:
#function for drawing bar chart for emotion preditions
def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()



# **Testing Random Image #1**

In [ ]:
#load some random image
from keras.preprocessing import image
img = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/Fuad_Assignment/jackman.png', grayscale=False, target_size=(48, 48) )#change image from google drive
#preprocess the random image

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x /= 255.
custom = new_model.predict(x)
emotion_analysis(custom[0])

#x = np.array(x, 'float32')
#x = x.reshape([48, 48]);

plt.gray()
plt.imshow(img)
plt.show()


# **Testing Random Image #2**

In [ ]:
#load some random image
from keras.preprocessing import image
img = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/Fuad_Assignment/sadspidey.jpg', grayscale=False, target_size=(48, 48) )#change image from google drive
#preprocess the random image

x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x /= 255.
custom = %timeit new_model.predict(x)
emotion_analysis(custom[0])

#x = np.array(x, 'float32')
#x = x.reshape([48, 48]);

plt.gray()
plt.imshow(img)
plt.show()

end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)